In [1]:
import imageio
import os
import warnings
from os import listdir
from os.path import isfile, join

from google.colab import drive
from google.colab import files

In [2]:
drive.mount('/content/drive')
base_path = '/content/drive/My Drive/COS Seesaw Research'

Mounted at /content/drive


In [3]:
cos_sites = ['alt', 'brw', 'cgo', 'hfm', 'kum', 'lef', 'mhd', 'mlo', 'nwr', 'psa', 'smo', 'spo', 'sum', 'thd' ]
warnings.filterwarnings("ignore")

In [4]:
# generate dates
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
date_start = dt(year=2016, month=1, day=1)
dates = []
for i in range(12):
  dates.append(str(date_start.date()))
  date_start = date_start + relativedelta(months=1)

In [ ]:
from PIL import Image, ImageFont, ImageDraw

for site in cos_sites:
  working_directory = base_path + '/Sensitivity/interval_correlation/' + site
  heatmap_path = working_directory + '/heatmaps/'
  shortlist_path = working_directory + '/shortlist/' + site + '_short_list_time.txt'
  gif_path = working_directory + '/shortlist/gifs'
  if not os.path.isdir(gif_path):
    os.mkdir(gif_path)

  shortlist_file = open(shortlist_path, 'r')
  top_features = {}
  # get list of unique features
  for i in range(10):
    line = shortlist_file.readline()
    entry = line.split(':')
    entry = entry[0]
    entry = entry.split(' ')[1]
    entry = entry.split('_')
    test_entry = entry[1].split('-')
    feature_name = None
    if len(test_entry) == 1:
      feature_name = entry[1] + '-current'
    else:
      feature_name = entry[1]

    if feature_name not in top_features.keys():
      top_features[feature_name] = []
    
    top_features[feature_name].append(entry[0])

  shortlist_file.close()

  for feature in top_features.keys():
    plots = []
    important_sites = 'Important Sites: ' + ', '.join(top_features[feature])
    # gif path
    # get all plots for the feature
    for date in dates:
      plot_name = heatmap_path + date + '--' + site +'_sensitivity_to_ ' + feature + '.png'

      plot = Image.open(plot_name).copy()
      image_editable = ImageDraw.Draw(plot)
      size = 18
      font = ImageFont.truetype('LiberationMono-Regular.ttf', size)
      x, y = plot.size
      y = y - size
      x = 1250
      #x = x - (size * len(important_sites))
      #print(x - (size *len(important_sites)))

      image_editable.text((x,y), important_sites, (55, 55, 55), font=font)
      #print(plot.size)
      #display(plot)

      plots.append(plot)
      #plots.append(image_editable)
      #display(image_editable)
    gif_name = gif_path + '/' + feature + '.gif'
    print(len(plots))
    plots[0].save(gif_name, save_all=True, append_images=plots[1:], optimize=False, duration=1500, loop=0)
    #imageio.mimsave(gif_name, plots)